In [1]:
from google.cloud import storage
from tqdm import tqdm
import json 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import geopandas
import random
from scipy.spatial import cKDTree

# Generate Object Detection Dataset

**Author:** Madhava Paliyam (madhavapaliyam@gmail.com)

**Description:** Converts the images that have been labeled so far into a dataset on google cloud for future training of the YOLO v5 model. 



**Inputs**: Train/Validation split ratio (ex. .2), distance threshold for close points (ex. .001 degrees ~100 m but depends on location)

**Outputs**: train.csv, val.csv updated in google cloud through dvc

This is done through the following steps: 

1. Obtain and parse all .jsonl files from labeled bucket 
2. Split the data into train and validation sets randomly stratified by country 
3. Filter the sets to ensure nearby points are in the same set 
4. Convert labels into YOLO format
5. Track updates to train.csv and val.csv in google cloud 

In [2]:
##### SET PARAMETERS HERE ######

TRAIN_VAL_SPLIT = .2        # proportion of validation/train images 
DISTANCE_THRESHOLD = .001   # min degrees of seperation between images in train and val 



!dvc pull -q -f

### 1. Obtain and Parse all .jsonl files from labeled bucket

In [4]:
# Initialize connections to cloud storage and database
client = storage.Client()

gcloud_labeling_bucket_str = 'street2sat-gcloud-labeling'
# get all jsonl objects
jsonl_blobs = [file for file in client.list_blobs(gcloud_labeling_bucket_str) if file.name.endswith(".jsonl")]

# get all info about all images
all_paths = pd.read_csv('gs://street2sat-database-csv/database-info.csv')

print(f"Found {len(jsonl_blobs)} jsonl files in the bucket.")

/home/paliyamm/.conda/envs/lacuna/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Found 5 jsonl files in the bucket.


In [5]:
# iterates through the jsonl files and creates two dataframes 
# image_info_df and bounding_box_df 
def get_image_and_bbox_df(blobs: list):
    images = []
    country_codes = []
    bounding_boxes = []
    lat = []
    lon = []
    for blob in blobs:
        str_rep = blob.download_as_string()
        for image_labels in tqdm(str_rep.splitlines()):
            # contains all the bounding box info and image path
            info_image = json.loads(image_labels)
            image_path = info_image['imageGcsUri']
            bounding_box_info = info_image['boundingBoxAnnotations']
            for b in bounding_box_info: 
                b['path'] = image_path

            # obtain image location from database and convert to tuple
            row = all_paths[all_paths['input_img'] == image_path]
            assert len(row) == 1, f'got {len(row)} instead of 1'
            row = row.iloc[0]

            if (row['cc'] == np.nan):
                # dont add image if no location is available 
                continue 

            lat.append(row['latitude'])
            lon.append(row['longitude'])
            country_codes.append(row['cc'])
            images.append(image_path)
            bounding_boxes.extend(bounding_box_info)
   
    df = pd.DataFrame()
    df['path'] = images
    df['cc'] = country_codes
    df['lat'] = lat 
    df['lon'] = lon
    df_bounding_boxes = pd.DataFrame(bounding_boxes)
    return df, df_bounding_boxes

image_info_df,bounding_box_df = get_image_and_bbox_df(jsonl_blobs)

100%|██████████| 99/99 [00:08<00:00, 11.55it/s]


### 2. Split the data into train and validation sets randomly stratified by country 

In [6]:
vc = image_info_df['cc'].value_counts()
countries = vc.keys()
print(vc)

UG    264
KE    119
US     93
Name: cc, dtype: int64


In [7]:
train_set = pd.DataFrame()
val_set = pd.DataFrame()
for country in countries:
    train, test = train_test_split(image_info_df[image_info_df['cc'] == country], test_size=TRAIN_VAL_SPLIT, shuffle = True)
    train_set = train_set.append(train)
    val_set = val_set.append(test)

# shuffle 
train_set = train_set.sample(frac = 1)
val_set = val_set.sample(frac = 1)

In [8]:
train_set.sample(10)

,path,cc,lat,lon
397,gs://street2sat-uploaded/Uganda/2021-06-25_Edr...,UG,0.214865,30.130855
138,gs://street2sat-uploaded/KENYA/2021-07-23-T1/G...,KE,-1.091433,34.442601
348,gs://street2sat-uploaded/Uganda/2021-06-29_Olw...,UG,2.561266,31.797074
254,gs://street2sat-uploaded/KENYA/2021-07-09-T1/G...,KE,-0.232417,35.530199
53,gs://street2sat-uploaded/Uganda/2021-06-23_Eas...,UG,0.943773,34.226561
195,gs://street2sat-uploaded/USA/2021-08-20-cropto...,US,41.117705,-84.495900
396,gs://street2sat-uploaded/Uganda/2021-06-29_Olw...,UG,2.553670,31.902539
137,gs://street2sat-uploaded/Uganda/1829244/2020-0...,UG,1.011017,34.331257
149,gs://street2sat-uploaded/Uganda/2021-06-25_Edr...,UG,0.211364,30.120597
162,gs://street2sat-uploaded/USA/2021-08-20-cropto...,US,40.600290,-88.201389


### 3. Filter the sets to ensure nearby points are in the same set 

In [9]:
'Returns distance and name of each neighbor in gdB from each point in gdA using a kd tree'
def ckdnearest(gdA, gdB):
    
    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf


'filters on closeby points and returns the new datasets'
random.seed(10)
def filter_closeby_points(train_set, val_set): 

    train_set_g = geopandas.GeoDataFrame(train_set.copy(), geometry=geopandas.points_from_xy(train_set.lon, train_set.lat, crs='EPSG:4326'))
    val_set_g = geopandas.GeoDataFrame(val_set.copy(), geometry=geopandas.points_from_xy(val_set.lon, val_set.lat, crs='EPSG:4326'))

    val_set_g.rename(columns = {'path':'val_path'}, inplace = True)
    train_set_g.rename(columns = {'path':'train_path'}, inplace = True)


    res = ckdnearest(val_set_g, train_set_g)


    print('Found {} closeby points.'.format(len(res[res['dist'] < DISTANCE_THRESHOLD])))

    # add images that are too close to each other to the same set randomly
    old_val_len = len(val_set)
    old_train_len = len(train_set)

    for i,r in res[res['dist'] < .001].iterrows():
        val_row = val_set[val_set['path'] == r['val_path']]
        train_row = train_set[train_set['path'] == r['train_path']]
        
        which = random.choice(['train', 'val'])
        if which == 'train':
            val_set = val_set.drop(val_row.index)
            train_set = pd.concat([train_set, val_row])
        elif which == 'val':
            train_set = train_set.drop(train_row.index)
            val_set = pd.concat([val_set, train_row])
        
        assert old_train_len - len(train_set) == len(val_set) - old_val_len, 'lengths are incorrect'

    
    

    return train_set, val_set, len(res[res['dist'] < DISTANCE_THRESHOLD])

In [10]:
# run till 0 closeby points are found 
closeby = -1
while closeby != 0:
    train_set, val_set, closeby = filter_closeby_points(train_set, val_set)

Found 7 closeby points.
Found 0 closeby points.


### 4. Convert labels into YOLO format

In [11]:
# converts google cloud labels to yolo format
def convert_to_yolo(xmin, ymin, xmax, ymax):
    xcenter = (xmax + xmin) / 2 
    ycenter = (ymax + ymin) / 2

    width = xmax - xmin 
    height = ymax - ymin 

    return xcenter, ycenter, width, height 

# open the classes to index dictionary
classes_dict = {}
with open('../street2sat_utils/crop_info/classes.txt') as classes_file: 
    for i, line in enumerate(classes_file):
        classes_dict[line.strip()] = i


# gets bounding boxes for each set in yolo format for the dataset
def get_bounding_boxes_for_df(dataset, bounding_box_df, classes_dict):
    bounding_box_list = []
    for i,image in tqdm(dataset.iterrows(), total = dataset.shape[0]):
        annotations_for_img = bounding_box_df[bounding_box_df['path'] == image['path']]
        boxes = ''
        for _,row in annotations_for_img.iterrows():
            xcenter, ycenter, width, height = convert_to_yolo(row['xMin'], row['yMin'], row['xMax'], row['yMax'])
            obj_class = classes_dict[row['displayName']]
            boxes += f'{obj_class} {xcenter} {ycenter} {width} {height}\n'

        bounding_box_list.append(boxes)

    return bounding_box_list


train_set['bounding_boxes'] = get_bounding_boxes_for_df(train_set, bounding_box_df, classes_dict)
val_set['bounding_boxes'] = get_bounding_boxes_for_df(val_set, bounding_box_df, classes_dict)


100%|██████████| 96/96 [00:00<00:00, 1710.74it/s]


In [12]:
train_set.sample(5)

,path,cc,lat,lon,bounding_boxes
288,gs://street2sat-uploaded/USA/2021-08-20-cropto...,US,40.487624,-88.855890,5 0.20853080568720378 0.6000000000000001 0.041...
248,gs://street2sat-uploaded/KENYA/2021-07-10-T1/G...,KE,-0.175233,35.581210,
232,gs://street2sat-uploaded/Uganda/2021-06-25_Edr...,UG,0.219991,30.126699,
101,gs://street2sat-uploaded/Uganda/2021-07-24_bod...,UG,0.530382,32.452628,
303,gs://street2sat-uploaded/KENYA/2021_07_08_T2/G...,KE,0.521671,35.093806,


### 5. Track updates to train.csv and val.csv in google cloud 
- Add and commit changes


In [13]:
# upload from data/train.csv and data/val.csv 
train_set.to_csv('../data/train.csv')
val_set.to_csv('../data/val.csv')

In [16]:
!dvc commit ../data/train.csv.dvc
!dvc commit ../data/val.csv.dvc
!dvc push ../data/train.csv.dvc
!dvc push ../data/val.csv.dvc

!git add ../data/train.csv.dvc
!git add ../data/val.csv.dvc
!git commit -m "changed train and val"
!git push

outputs ['../data/train.csv'] of stage: '../data/train.csv.dvc' changed. Are you sure you want to commit it? [y/n] ^C
ERROR: interrupted by the user
outputs ['../data/val.csv'] of stage: '../data/val.csv.dvc' changed. Are you sure you want to commit it? [y/n] ^C
ERROR: interrupted by the user
Everything is up to date.                                                       
Everything is up to date.                                                       
# On branch analyze-model-results
# Changes not staged for commit:
#   (use "git add/rm <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	deleted:    1_create_dataset_crop_or_region.ipynb
#	deleted:    1_create_dataset_random_images.ipynb
#	modified:   2_generate_object_detection_dataset.ipynb
#	deleted:    3_train_and_validate_YOLO.ipynb
#
# Untracked files:
#   (use "git add <file>..." to include in what will be committed)
#
#	3_train_YOLO.ipynb
#	[WIP]1_create